<a href="https://colab.research.google.com/github/Aaravrao5150/Forest-Fire-Detection-Using-Satellite-Imagery/blob/main/Forest_Fire_Detection_Using_Satellite_Imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align: center;">Forest Fire Detection using Computer Vision</h1>


## 1. Problem Statement

Imagine you are in charge of watching a big forest. You want to keep everyone safe, but sometimes wildfires can start without warning. These fires can spread quickly and cause a lot of damage. So, we want to build a computer program that can look at pictures of forests and tell us if there’s a fire starting or not. This program can help people react faster to stop wildfires!

Ensure installation of this dataset
  https://www.kaggle.com/datasets/abdelghaniaaba/wildfire-prediction-dataset

## 2. Objectives

This project will:

    Teach the computer to recognize pictures with wildfires and without wildfires.
    Build a model (a smart program) that looks at forest pictures and decides if there’s a fire.
    Show us a simple screen (a GUI) where we can upload forest pictures and get quick answers on fire danger.
    Help keep forests and people safe by giving early warnings about fires.

### 3. Dataset Information

To teach the computer about wildfires, we’ll use a special set of pictures called a dataset. This dataset has pictures of forests divided into two types: with a fire (wildfire) and without a fire (no wildfire). These images help the computer learn to tell the difference.

    Where to Get the Dataset: You can download the dataset from Kaggle
    https://www.kaggle.com/datasets/abdelghaniaaba/wildfire-prediction-dataset
    Dataset Folders:
        Inside, there are three main folders: train, test, and valid.
        Each folder has two subfolders:
            Wildfire: Contains pictures showing wildfires in forests.
            NoWildfire: Contains pictures showing forests without any fire.

## 4. Understanding the CNN Model

CNN stands for Convolutional Neural Network. It’s a special type of program that can look at pictures and learn to find important details. Imagine how you look at a picture and notice trees, animals, or fire — CNN does something similar!

Here’s how it works, step-by-step:

    Looking at Patterns: The CNN looks at lots of tiny patterns in pictures, like shapes and colors.
    Learning from Examples: It looks at many images of wildfires and no wildfires to learn what makes them different.
    Making Predictions: Once trained, the CNN can look at a new picture and guess if it shows a fire or not.

## 5. Code Implementation

### Step 1: Importing Libraries

We’ll use Python with TensorFlow/Keras for the CNN model and Tkinter for the GUI. Make sure you have these libraries installed.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for data augumentation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog #provides a dialog box to select files.
from PIL import Image, ImageTk #ImageTk - Converts images for display in Tkinter GUIs
import numpy as np

import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.45G/1.45G [00:58<00:00, 26.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1


### Step 2: Loading and Preprocessing the Data

We'll use ImageDataGenerator to load images from your dataset's train, valid, and test folders. This will resize and normalize images so our model can use them.

In [ ]:
# Set up directories
train_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train"
valid_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid"
test_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test"

# Set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')


Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


### Step 3: Building the CNN Model

The CNN model will have multiple convolutional layers to extract features, followed by dense layers for classification.
- Sequential: When you need a simple feed-forward model with a single input and output.
- Functional: When your model has multiple inputs/outputs or non-linear architectures.
- Subclassing: When custom forward logic is required or your model's architecture cannot be easily represented using pre-defined layers.

In [ ]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire or no wildfire
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


We noticed some images in our dataset are incomplete or corrupted, causing TensorFlow to fail when trying to load them.
Here are a few steps to handle this issue:

Solution: Skip Corrupted Images

One way to handle this is to modify the data loading process to skip over corrupted images.

    Re-import the PIL library with a setting that will ignore truncated images.

    We add this line at the beginning of your script.

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### Step 4: Training the Model

Train the model with the train and valid data.

In [ ]:
# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=10, verbose=1)

In [ ]:
#save the model
model.save("ffd_model.h5")

### Step 5: Building the GUI with Tkinter

Now, let’s create a GUI that lets us upload an image and predict if there’s a wildfire.

In [ ]:
import tkinter as tk
from tkinter import filedialog

from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model

In [ ]:
# Load the model
model = load_model("ffd_model.h5")
print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
# Install gradio if not already installed
!pip install gradio

import gradio as gr
import tensorflow as tf
from PIL import Image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('ffd_model.h5')

# Function to preprocess and predict the image
def predict_image(image):
    # Resize the image to match the model input
    img_for_model = image.resize((64, 64))
    img_array = np.array(img_for_model) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make a prediction
    prediction = model.predict(img_array)[0][0]  # Extract the prediction
    result = "Wildfire" if prediction > 0.5 else "No Wildfire"
    return result

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_image,  # The function that makes predictions
    inputs=gr.Image(type="pil"),  # Input is an image file (PIL format)
    outputs="text",  # Output is text (prediction result)
    live=True  # Updates live as image is uploaded
)

# Launch the interface
iface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://89a5eea152fc0dc1e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Step 6: Testing the Model

You can now use the GUI to test the model. Click "Upload Image," select an image, and the GUI will display the image and predict if it shows a wildfire or not.

## 5. Conclusion
This project demonstrates the application of computer vision, specifically using Convolutional Neural Networks (CNNs), to detect wildfires in forest images. By training a CNN model on a dataset of wildfire and non-wildfire images, we developed a system capable of identifying potential fire threats. This technology has the potential to significantly improve wildfire detection and response, contributing to the safety of both people and natural environments.